In [10]:
import tensorflow as tf
import os
from pathlib import Path
import urllib.request as request
import numpy as np
import matplotlib.pyplot as plt

In [3]:
IMAGE_SIZE = (224, 224)

In [5]:
model = tf.keras.applications.VGG16(
    include_top=True, weights="imagenet", input_tensor=None, input_shape=None, classes=1000 
)    ## include_top = True, meanse include FC layers as well

In [6]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
model.save("VGG16_full_model.h5")

In [8]:
model.layers[1].padding

'same'

In [5]:
data_URL = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

# create a directory
def create_dirs(dir_path):
    os.makedirs(dir_path, exist_ok=True)
    print(f"{dir_path} directory created")
    
ROOT_DATA_DIR = "data"
create_dirs(ROOT_DATA_DIR)


data_zip_file = "data.zip"
data_zip_path = os.path.join(ROOT_DATA_DIR, data_zip_file)

if not os.path.isfile(data_zip_file):
    print("downloading data...")
    filename, headers = request.urlretrieve(data_URL, data_zip_path)
    print(f"filename: {filename} created with info \n{headers}")
else:
    print(f"file is already present")

data directory created
downloading data...
filename: data\data.zip created with info 
Content-Type: application/zip
Content-Length: 47286322
Connection: close
Last-Modified: Wed, 15 Mar 2017 18:46:00 GMT
x-amz-version-id: null
Accept-Ranges: bytes
Server: AmazonS3
Date: Sat, 02 Apr 2022 08:09:46 GMT
ETag: "5f8c32a6554f6acb4d649776e7735e48"
X-Cache: Hit from cloudfront
Via: 1.1 b93822242d240fe957b16155421ce866.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: NRT57-P2
X-Amz-Cf-Id: oDfh5X-WA9UxgaP2Y0WYyWpHo7QyGPJANZI2rCwhJ6wXY1GARabaIg==
Age: 68796




In [6]:
from zipfile import ZipFile

unzip_data_dirname = "unzip_data_dir"
unzip_data_dir = os.path.join(ROOT_DATA_DIR, unzip_data_dirname)

if not os.path.exists(unzip_data_dir):
    os.makedirs(unzip_data_dir, exist_ok=True)
    with ZipFile(data_zip_path) as f:
        f.extractall(unzip_data_dir)
else:
    print(f"data already extacted")

In [7]:
main_data_dir = Path("./data/unzip_data_dir/hymenoptera_data/train")
main_data_dir

WindowsPath('data/unzip_data_dir/hymenoptera_data/train')

In [8]:
BATCH_SIZE = 32
pixels = 224
IMAGE_SIZE = (pixels, pixels)

In [9]:
datagen_kwargs = dict(rescale=1./255, validation_split=0.20)

dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)

In [10]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)


valid_generator = valid_datagen.flow_from_directory(
    main_data_dir, subset="validation", shuffle=False, **dataflow_kwargs)


do_data_augmentation = False

if do_data_augmentation:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotational_range=40,
        horizontal_flip=True,
        width_shift_range=0.2, 
        height_shift_range=0.2, 
        shear_range=0.2, 
        zoom_range=0.2, 
        **datagen_kwargs)
else:
    train_datagen = valid_datagen

train_generator = train_datagen.flow_from_directory(
    main_data_dir, subset="training", shuffle=True, **dataflow_kwargs
)

Found 48 images belonging to 2 classes.
Found 196 images belonging to 2 classes.


In [11]:
RGB_IMAGE_SIZE = (pixels, pixels, 3)


vgg = tf.keras.applications.vgg16.VGG16(
    input_shape=RGB_IMAGE_SIZE,
    weights="imagenet",
    include_top=False   ## do not include the FC layer
)

58900480/58889256 [==============================] - 3s 0us/step


In [12]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")

input_2             : True
block1_conv1        : True
block1_conv2        : True
block1_pool         : True
block2_conv1        : True
block2_conv2        : True
block2_pool         : True
block3_conv1        : True
block3_conv2        : True
block3_conv3        : True
block3_pool         : True
block4_conv1        : True
block4_conv2        : True
block4_conv3        : True
block4_pool         : True
block5_conv1        : True
block5_conv2        : True
block5_conv3        : True
block5_pool         : True


In [14]:
for layer in vgg.layers:
    layer.trainable = False

In [15]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")

input_2             : False
block1_conv1        : False
block1_conv2        : False
block1_pool         : False
block2_conv1        : False
block2_conv2        : False
block2_pool         : False
block3_conv1        : False
block3_conv2        : False
block3_conv3        : False
block3_pool         : False
block4_conv1        : False
block4_conv2        : False
block4_conv3        : False
block4_pool         : False
block5_conv1        : False
block5_conv2        : False
block5_conv3        : False
block5_pool         : False


In [16]:
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [17]:
del model

In [18]:
CLASSES = 2
x = tf.keras.layers.Flatten()(vgg.output)
prediction = tf.keras.layers.Dense(CLASSES, activation="softmax")(x)

model = tf.keras.models.Model(inputs=vgg.input, outputs = prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [20]:
EPOCHS = 10

history = model.fit(
    train_generator, epochs=EPOCHS, validation_data=valid_generator
)

Epoch 1/10
7/7 [==============================] - 48s 7s/step - loss: 10.7963 - accuracy: 0.5663 - val_loss: 31.4450 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 65s 9s/step - loss: 29.3667 - accuracy: 0.5357 - val_loss: 35.7914 - val_accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - 68s 10s/step - loss: 28.8905 - accuracy: 0.5969 - val_loss: 35.0689 - val_accuracy: 0.5000
Epoch 4/10
7/7 [==============================] - 71s 10s/step - loss: 24.3502 - accuracy: 0.6122 - val_loss: 47.9907 - val_accuracy: 0.5000
Epoch 5/10
7/7 [==============================] - 64s 10s/step - loss: 21.5244 - accuracy: 0.6531 - val_loss: 11.7447 - val_accuracy: 0.7500
Epoch 6/10
7/7 [==============================] - 73s 11s/step - loss: 10.5099 - accuracy: 0.7398 - val_loss: 3.9106 - val_accuracy: 0.8125
Epoch 7/10
7/7 [==============================] - 65s 9s/step - loss: 10.0786 - accuracy: 0.7602 - val_loss: 24.4857 - val_accuracy: 0.6042
Epoch 8/10
7/7 [=